<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/torchtune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/pytorch/torchtune

In [1]:
!nvidia-smi

Sun Apr 28 20:50:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   34C    P8              11W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install torchtune -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 17.7 MB/s eta 0:00:00
  Preparing metadata

In [3]:
!tune -h

usage: tune [-h] {download,ls,cp,run,validate} ...

Welcome to the TorchTune CLI!

options:
  -h, --help            show this help message and exit

subcommands:
  {download,ls,cp,run,validate}
    download            Download a model from the Hugging Face Hub.
    ls                  List all built-in recipes and configs
    cp                  Copy a built-in recipe or config to a local path.
    run                 Run a recipe. For distributed recipes, this supports all torchrun
                        arguments.
    validate            Validate a config and ensure that it is well-formed.


In [4]:
#meta-llama/Llama-2-7b-hf

In [5]:
!pip install colab-env --quiet
!pip install huggingface_hub -q

import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

  Preparing metadata (setup.py) ... done
Mounted at /content/gdrive


In [6]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
#!tune download meta-llama/Llama-2-7b-hf \
#--output-dir /tmp/Llama-2-7b-hf \
#--hf-token <HF_TOKEN> \

'HF_TOKEN' is in my Enviroment

In [8]:
!which tune

/usr/local/bin/tune


In [9]:
#%mkdir -p /tmp/Llama-2-7b-hf
!tune download meta-llama/Llama-2-7b-hf --output-dir /tmp/Llama-2-7b-hf

Ignoring files matching the following patterns: *.safetensors
Fetching 15 files:   0% 0/15 [00:00<?, ?it/s]
config.json: 100% 609/609 [00:00<00:00, 4.24MB/s]

LICENSE.txt: 100% 7.02k/7.02k [00:00<00:00, 43.1MB/s]

model.safetensors.index.json:   0% 0.00/26.8k [00:00<?, ?B/s]

README.md:   0% 0.00/22.3k [00:00<?, ?B/s]


USE_POLICY.md: 100% 4.77k/4.77k [00:00<00:00, 10.4MB/s]
README.md: 100% 22.3k/22.3k [00:00<00:00, 14.1MB/s]
model.safetensors.index.json: 100% 26.8k/26.8k [00:00<00:00, 10.8MB/s]

Responsible-Use-Guide.pdf:   0% 0.00/1.25M [00:00<?, ?B/s]

.gitattributes: 100% 1.58k/1.58k [00:00<00:00, 11.8MB/s]
Responsible-Use-Guide.pdf: 100% 1.25M/1.25M [00:00<00:00, 15.7MB/s]

generation_config.json: 100% 188/188 [00:00<00:00, 1.70MB/s]

tokenizer.json: 100% 1.84M/1.84M [00:00<00:00, 46.9MB/s]

pytorch_model.bin.index.json: 100% 26.8k/26.8k [00:00<00:00, 78.7MB/s]

special_tokens_map.json: 100% 414/414 [00:00<00:00, 2.84MB/s]

pytorch_model-00002-of-00002.bin:   0% 0.00/3.50G [00:00<

In [10]:
!ls -ltha /tmp/Llama-2-7b-hf/*

lrwxrwxrwx 1 root root  138 Apr 28 20:55 /tmp/Llama-2-7b-hf/pytorch_model-00001-of-00002.bin -> ../../root/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-hf/blobs/ee62ed2ad7ded505ae47df50bc6c52916860dfb1c009df4715148cc4bfb50d2f
lrwxrwxrwx 1 root root  138 Apr 28 20:54 /tmp/Llama-2-7b-hf/pytorch_model-00002-of-00002.bin -> ../../root/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-hf/blobs/1fd7762035b3ca4f2d6af6bf10129689a119b7c38058025f9842511532ea02fb
-rw-r--r-- 1 root root 489K Apr 28 20:54 /tmp/Llama-2-7b-hf/tokenizer.model
-rw-r--r-- 1 root root  776 Apr 28 20:54 /tmp/Llama-2-7b-hf/tokenizer_config.json
-rw-r--r-- 1 root root  414 Apr 28 20:54 /tmp/Llama-2-7b-hf/special_tokens_map.json
-rw-r--r-- 1 root root  27K Apr 28 20:54 /tmp/Llama-2-7b-hf/pytorch_model.bin.index.json
-rw-r--r-- 1 root root 1.8M Apr 28 20:54 /tmp/Llama-2-7b-hf/tokenizer.json
-rw-r--r-- 1 root root  188 Apr 28 20:54 /tmp/Llama-2-7b-hf/generation_config.json
-rw-r--r-- 1 root root 1.2M Apr 28 20:54 

QLoRA has about 75% smaller peak GPU memory usage compared to LoRA. LoRA is about 66% faster than QLoRA in terms of tuning speed. While both methods are relatively inexpensive, LoRA is up to 40% less expensive than QLoRA. Higher max sequence length increases GPU memory consumption.

In [11]:
!tune ls

#RECIPE                                   CONFIG
#full_finetune_single_device              llama2/7B_full_low_memory
#                                         mistral/7B_full_low_memory
#full_finetune_distributed                llama2/7B_full
#                                         llama2/13B_full
#                                         mistral/7B_full
#lora_finetune_single_device              llama2/7B_lora_single_device
#                                         llama2/7B_qlora_single_device
#                                         mistral/7B_lora_single_device

RECIPE                                   CONFIG                                  
full_finetune_single_device              llama2/7B_full_low_memory               
                                         llama3/8B_full_single_device            
                                         mistral/7B_full_low_memory              
full_finetune_distributed                llama2/7B_full                          
                                         llama2/13B_full                         
                                         llama3/8B_full                          
                                         mistral/7B_full                         
                                         gemma/2B_full                           
lora_finetune_single_device              llama2/7B_lora_single_device            
                                         llama2/7B_qlora_single_device           
                                         llama3/8B_lora_single_device            
                

In [12]:
#ERROR WITH T4 GPU:  RuntimeError: bf16 precision was requested but not available on this hardware. Please use fp32 precision instead.

https://pytorch.org/torchtune/stable/_modules/torchtune/utils/precision.html

https://pytorch.org/torchtune/stable/tutorials/first_finetune_tutorial.html


https://pytorch.org/torchtune/stable/tutorials/e2e_flow.html#e2e-flow


In [13]:
!tune run lora_finetune_single_device --config llama2/7B_lora_single_device

INFO:torchtune.utils.logging:Running LoRAFinetuneRecipeSingleDevice with resolved config:

batch_size: 2
checkpointer:
  _component_: torchtune.utils.FullModelHFCheckpointer
  adapter_checkpoint: null
  checkpoint_dir: /tmp/Llama-2-7b-hf
  checkpoint_files:
  - pytorch_model-00001-of-00002.bin
  - pytorch_model-00002-of-00002.bin
  model_type: LLAMA2
  output_dir: /tmp/Llama-2-7b-hf
  recipe_checkpoint: null
compile: false
dataset:
  _component_: torchtune.datasets.alpaca_cleaned_dataset
  train_on_input: true
device: cuda
dtype: bf16
enable_activation_checkpointing: true
epochs: 1
gradient_accumulation_steps: 64
log_every_n_steps: null
loss:
  _component_: torch.nn.CrossEntropyLoss
lr_scheduler:
  _component_: torchtune.modules.get_cosine_schedule_with_warmup
  num_warmup_steps: 100
max_steps_per_epoch: null
metric_logger:
  _component_: torchtune.utils.metric_logging.DiskLogger
  log_dir: /tmp/lora_finetune_output
model:
  _component_: torchtune.models.llama2.lora_llama2_7b
  apply_l

In [14]:
#[_checkpointer.py:473] Model checkpoint of size 9.98 GB saved to <checkpoint_dir>/hf_model_0001_0.pt

#[_checkpointer.py:473] Model checkpoint of size 3.50 GB saved to <checkpoint_dir>/hf_model_0002_0.pt

#[_checkpointer.py:484] Adapter checkpoint of size 0.01 GB saved to <checkpoint_dir>/adapter_0.pt

#checkpoint_dir='/tmp/Llama-2-7b-hf/'
#/tmp/Llama-2-7b-hf/tokenizer.model
#INFO:torchtune.utils.logging:Model checkpoint of size 9.98 GB saved to /tmp/Llama-2-7b-hf/hf_model_0001_0.pt
#INFO:torchtune.utils.logging:Model checkpoint of size 3.50 GB saved to /tmp/Llama-2-7b-hf/hf_model_0002_0.pt
#INFO:torchtune.utils.logging:Adapter checkpoint of size 0.01 GB saved to /tmp/Llama-2-7b-hf/adapter_0.pt

In [ ]:
#!tune cp eleuther_evaluation ./custom_eval_config.yaml \

!tune cp eleuther_evaluation /content/custom_eval_config.yaml



https://huggingface.co/frankmorales2020/Llama-2-7b-hf-text-to-sql-flash-attention-2


huggingface-cli upload 'hf-repo-id' 'checkpoint-dir'

https://huggingface.co/'hf-repo-id'/tree/main/.



In [57]:
!cat /content/custom_eval_config.yaml

# Config for EleutherEvalRecipe in eleuther_eval.py
#
# To launch, run the following command from root torchtune directory:
#    tune run eleuther_eval --config eleuther_evaluation tasks=["truthfulqa_mc2","hellaswag"]

# Model Arguments
model:
  _component_: torchtune.models.llama2.llama2_7b

checkpointer:
  _component_: torchtune.utils.FullModelHFCheckpointer
  checkpoint_dir: /tmp/Llama-2-7b-hf
  checkpoint_files: [
    pytorch_model-00001-of-00002.bin,
    pytorch_model-00002-of-00002.bin,
  ]
  recipe_checkpoint: null
  output_dir: /tmp/Llama-2-7b-hf
  model_type: LLAMA2

# Tokenizer
tokenizer:
  _component_: torchtune.models.llama2.llama2_tokenizer
  path: /tmp/Llama-2-7b-hf/tokenizer.model

# Environment
device: cuda
dtype: bf16
seed: 217

# EleutherAI specific eval args
tasks: ["truthfulqa_mc2"]
limit: null
max_seq_length: 4096

# Quantization specific args
quantizer: null


In [17]:
!pip install lm_eval==0.4.* -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.1/111.1 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 34.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


BEFORE tunning

2024-04-29:03:49:16,837 INFO     [eleuther_eval.py:198] truthfulqa_mc2: {'acc,none': 0.3891793196860692, 'acc_stderr,none': 0.013564855356631, 'alias': 'truthfulqa_mc2'}

In [ ]:
#RECIPE                                   CONFIG
#eleuther_eval                            eleuther_evaluation

#/tmp/Mistral-7B-v0.1/tokenizer_config.json
#/tmp/Mistral-7B-v0.1/tokenizer.model
# /usr/local/lib/python3.10/dist-packages/torchtune/models/mistral
# /usr/local/lib/python3.10/dist-packages/torchtune/models/llama2

!tune run eleuther_eval --config /content/custom_eval_config.yaml

#checkpointer.checkpoint_dir='/tmp/Llama-2-7b-hf/' \
#tokenizer.path='/tmp/Llama-2-7b-hf'/tokenizer.model

#[evaluator.py:324] Running loglikelihood requests
#[eleuther_eval.py:195] Eval completed in 121.27 seconds.
#[eleuther_eval.py:197] truthfulqa_mc2: {'acc,none': 0.388...

In [60]:
!cat /content/custom_eval_config.yaml

# Config for EleutherEvalRecipe in eleuther_eval.py
#
# To launch, run the following command from root torchtune directory:
#    tune run eleuther_eval --config eleuther_evaluation tasks=["truthfulqa_mc2","hellaswag"]

# Model Arguments
model:
  _component_: torchtune.models.llama2.llama2_7b

checkpointer:
  _component_: torchtune.utils.FullModelHFCheckpointer
  checkpoint_dir: /tmp/Llama-2-7b-hf
  # checkpoint files for the fine-tuned model. This should
  # match what's shown in the logs above
  checkpoint_files: [
        hf_model_0001_0.pt,
        hf_model_0002_0.pt,
  ]

  recipe_checkpoint: null
  output_dir: /tmp/Llama-2-7b-hf
  model_type: LLAMA2

# Tokenizer
tokenizer:
  _component_: torchtune.models.llama2.llama2_tokenizer
  path: /tmp/Llama-2-7b-hf/tokenizer.model

# Environment
device: cuda
dtype: bf16
seed: 217

# EleutherAI specific eval args
tasks: ["truthfulqa_mc2"]
limit: null
max_seq_length: 4096

# Quantization specific args
quantizer: null


AFTER tunning

2024-04-29:03:58:45,703 INFO     [eleuther_eval.py:198] truthfulqa_mc2: {'acc,none': 0.4786091749231919, 'acc_stderr,none': 0.014549538233703927, 'alias': 'truthfulqa_mc2'}

In [ ]:
!tune run eleuther_eval --config /content/custom_eval_config.yaml

In [ ]:
!tune cp generation /content/custom_generation_config.yaml

In [63]:
!cat /content/custom_generation_config.yaml

# Config for running the InferenceRecipe in generate.py to generate output from an LLM
#
# To launch, run the following command from root torchtune directory:
#    tune run generate --config generation

# Model arguments
model:
  _component_: torchtune.models.llama2.llama2_7b

checkpointer:
  _component_: torchtune.utils.FullModelHFCheckpointer
  checkpoint_dir: /tmp/Llama-2-7b-hf/
  # checkpoint files for the fine-tuned model. This should
    # match what's shown in the logs above
    checkpoint_files: [
        hf_model_0001_0.pt,
        hf_model_0002_0.pt,
    ]
  
  output_dir: /tmp/Llama-2-7b-hf/
  model_type: LLAMA2

device: cuda
dtype: bf16

seed: 1234

# Tokenizer arguments
tokenizer:
  _component_: torchtune.models.llama2.llama2_tokenizer
  path: /tmp/Llama-2-7b-hf/tokenizer.model

# Generation arguments; defaults taken from gpt-fast
prompt: "Hello, my name is"
max_new_tokens: 300
temperature: 0.6 # 0.8 and 0.6 are popular values to try
top_k: 300

quantizer: null


In [ ]:
!tune run generate --config /content/custom_generation_config.yaml prompt="What are some interesting sites to visit in the Bay Area?"

2024-04-29:04:03:28,328 INFO     [generate.py:123] What are some interesting sites to visit in the Bay Area?
What are some interesting sites to visit in the Bay Area?
The Bay Area is home to many interesting sites, from the iconic Golden Gate Bridge to the quirky Alcatraz Island. Here are some of the most interesting sites to visit in the Bay Area:
Golden Gate Bridge: This suspension bridge is one of the most recognizable landmarks in the world. It spans the Golden Gate Strait, connecting San Francisco to Marin County. Visitors can take a walk or bike ride across the bridge and enjoy the stunning views of the bay.
Alcatraz Island: This former prison is now a popular tourist attraction. Visitors can take a ferry to the island and explore the cell blocks, hospital, and other buildings. There is also a museum on the island, which tells the history of the prison and its most famous inmates.
Coit Tower: This 210-foot tower is located in the Telegraph Hill neighborhood of San Francisco. It offers panoramic views of the city and the bay. Visitors can take an elevator to the top of the tower and enjoy the views.
Sutro Baths: These ruins are located in the Lands End area of San Francisco. The baths were a popular swimming and spa destination in the late 19th century, but closed in 1966. Today, visitors can explore the ru

2024-04-29:04:03:28,330 INFO     [generate.py:136] Time for inference: 20.19 sec total, 14.86 tokens/sec

2024-04-29:04:03:28,330 INFO     [generate.py:139] Bandwidth achieved: 233.11 GB/s

2024-04-29:04:03:28,331 INFO     [generate.py:140] Memory used: 15.72 GB


In [ ]:
# https://huggingface.co/frankmorales2020
!huggingface-cli upload torchtune-Llama-2-7b /tmp/Llama-2-7b-hf

https://huggingface.co/frankmorales2020/torchtune-Llama-2-7b